In [1]:
import ipywidgets as widgets
from IPython.display import display

In [2]:
buyer_sliders = {
    key: widgets.IntSlider(
        description='{unit} {key}'.format(unit=unit, key=key),
        value=default,
        )
    for unit, default, key in [
        ('$', 90, 'high'),
        ('$/min', 5, 'buyer_time_price'),
        ('$', 10, 'exp_low'),
        ('$/min', 5, 'exp_seller_time_price'),
        ('$', 50, 'buyer_exp_market'),
        ('min', 4, 'buy_time_fungibility'),
        ]
    }

In [3]:
seller_sliders = {
    key: widgets.IntSlider(
        description='{unit} {key}'.format(unit=unit, key=key),
        value=default,
        )
    for unit, default, key in [
        ('$', 10, 'low'),
        ('$/min', 5, 'seller_time_price'),
        ('$', 90, 'exp_high'),
        ('$/min', 5, 'exp_buyer_time_price'),
        ('$', 50, 'seller_exp_market'),
        ('min', 4, 'sell_time_fungibility'),
        ]
    }

In [4]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [5]:
# https://stackoverflow.com/questions/32000934/python-print-a-variables-name-and-value
import sys
DEBUG = False
def debug(expression):
    if DEBUG:
        frame = sys._getframe(1)
        print(expression, ':\t', repr(eval(expression, frame.f_globals, frame.f_locals)))

In [6]:
def simulsay(buyer, seller):
    print('{}\tB | S\t{}'.format(buyer, seller))

def narrate(text):
    print('\t{}'.format(text))

In [7]:
def report_sale(prices, elapsed_time, buyer_time_price, seller_time_price, high, low):
    try:
        final_price = mean(prices)
    except TypeError: # a single value
        final_price = prices
    debug('final_price') # $
    narrate('Agreed! Sold at ${}.'.format(final_price))
    narrate('Elapsed time: {} min'.format(elapsed_time))
    narrate('Final gross utilities:')
    simulsay(
        buyer='${}'.format(high-final_price),
        seller='${}'.format(final_price-low),
        )
    narrate('Final utilities net of time:')
    simulsay(
        buyer='${}'.format(high-final_price-(buyer_time_price*elapsed_time)),
        seller='${}'.format(final_price-low-(seller_time_price*elapsed_time)),
        )

def report_no_deal(buyer_null_util, seller_null_util, elapsed_time, buyer_time_price, seller_time_price):
    narrate('No deal. Gross null utilities:')
    simulsay(buyer=buyer_null_util, seller=seller_null_util)
    narrate('Null utilities net of time:')
    simulsay(
        buyer=buyer_null_util-(buyer_time_price*elapsed_time),
        seller=seller_null_util-(seller_time_price*elapsed_time),
        )
    
def calculate(high, buyer_time_price, exp_low, exp_seller_time_price, buyer_exp_market, buy_time_fungibility, low, seller_time_price, exp_high, exp_buyer_time_price, seller_exp_market, sell_time_fungibility):
    TIME_CONSTANT = 1 # min
    
    buyer_null_util = max(0, high - buyer_exp_market - (buyer_time_price * buy_time_fungibility))
    debug('buyer_null_util') # $
    seller_null_util = max(low, seller_exp_market - low - (seller_time_price * sell_time_fungibility))
    debug('seller_null_util') # $
    narrate('Null utilities:')
    simulsay(buyer=buyer_null_util, seller=seller_null_util)
    
    # ladies and gentlemen here's the opening round
    t = 0
    
    seller_bid = [exp_high]
    buyer_bid = [exp_low]
    debug('seller_bid[0]') # $
    debug('buyer_bid[0]') # $
    #seller_say('Selling at ${}.'.format(seller_bid[0]))
    #buyer_say('Buying at ${}.'.format(buyer_bid[0]))
    simulsay(
        seller='Selling at ${}.'.format(seller_bid[0]),
        buyer='Buying at ${}.'.format(buyer_bid[0]),
        )
    
    if buyer_bid[t] >= seller_bid[t]: # CHANGE: allow equal
        report_sale(
            prices=[buyer_bid[t], seller_bid[t]],
            elapsed_time=t,
            buyer_time_price=buyer_time_price,
            seller_time_price=seller_time_price,
            high=high,
            low=low,
            )
        return
    else:
        gap_between_opening_prices = seller_bid[t] - buyer_bid[t]
        debug('gap_between_opening_prices') # $
        
    # second round, let's see if anyone accepts
    # t is still 0 (representing where we came from)
    buyer_acc_imm_util = high - seller_bid[t]
    seller_acc_imm_util = buyer_bid[t] - low
    debug('buyer_acc_imm_util') # $
    debug('seller_acc_imm_util') # $
    
    # check if the buyer accepts
    if (
            (buyer_time_price * TIME_CONSTANT) > gap_between_opening_prices  # CHANGED: corrected by TIME_CONSTANT for units match
            and buyer_acc_imm_util > buyer_null_util
            ):
        buyer_text = 'I accept your price!'
        buyer_accepts = True
    else:
        buyer_accepts = False
        buyer_bid.append(buyer_bid[t] + (exp_seller_time_price * TIME_CONSTANT))
        debug('buyer_bid[{}]'.format(t+1))
        buyer_text = 'Buying at ${}.'.format(buyer_bid[t+1])
        
    
    # check if the seller accepts
    if (
            (seller_time_price * TIME_CONSTANT) > gap_between_opening_prices # CHANGED: corrected by TIME_CONSTANT for units match
            and seller_acc_imm_util > seller_null_util
            ):
        seller_text = 'I accept your price!'
        seller_accepts = True
    else:
        seller_accepts = False
        seller_bid.append(seller_bid[t] - (exp_buyer_time_price * TIME_CONSTANT))
        debug('seller_bid[{}]'.format(t+1))
        seller_text = 'Selling at ${}.'.format(seller_bid[t+1])
        
    simulsay(
        buyer=buyer_text,
        seller=seller_text,
        )
    
    if buyer_accepts and not seller_accepts:
        report_sale(
            prices=seller_bid[t],
            elapsed_time=t+1,
            buyer_time_price=buyer_time_price,
            seller_time_price=seller_time_price,
            high=high,
            low=low,
            )
        return
    elif seller_accepts and not buyer_accepts:
        report_sale(
            prices=buyer_bid[t],
            elapsed_time=t+1,
            buyer_time_price=buyer_time_price,
            seller_time_price=seller_time_price,
            high=high,
            low=low,
            )
        return
    elif buyer_accepts and seller_accepts:
        report_sale(
            prices=[buyer_bid[t], seller_bid[t]],
            elapsed_time=t+1,
            buyer_time_price=buyer_time_price,
            seller_time_price=seller_time_price,
            high=high,
            low=low,
            )
        return
    
    # round 3: we now know their guess of our time price
    t += 1 # t is now 1 (again representing where we came from)
    buyer_acc_imm_util = high - seller_bid[t]
    seller_acc_imm_util = buyer_bid[t] - low
    debug('buyer_acc_imm_util') # $
    debug('seller_acc_imm_util') # $
    
    # check if the buyer accepts
    # is our time underestimated?
    if exp_buyer_time_price < buyer_time_price: # theoretically: use their price step rather than their expectation, in case they're playing weird and reporting their expectation wrong
        # then acc/rej now, we won't get better
        if buyer_acc_imm_util > buyer_null_util:
            buyer_text = 'I accept your price!'
            buyer_accepts = True
            buyer_rejects = False
        else:
            buyer_accepts = False
            buyer_rejects = True
            buyer_text = 'You try my patience!'
    else:
        # we want to haggle
        buyer_accepts = False
        buyer_rejects = False
        buyer_text = 'Let\'s haggle.'
    
    # check if the seller accepts
    # is our time underestimated?
    if exp_seller_time_price < seller_time_price: # theoretically: use their price step rather than their expectation, in case they're playing weird and reporting their expectation wrong
        # then acc/rej now, we won't get better
        if seller_acc_imm_util > seller_null_util:
            seller_text = 'I accept your price!'
            seller_accepts = True
            seller_rejects = False
        else:
            seller_accepts = False
            seller_rejects = True
            seller_text = 'You try my patience!'
    else:
        # we want to haggle
        seller_accepts = False
        seller_rejects = False
        seller_text = 'Let\'s haggle.'
    
    simulsay(buyer=buyer_text, seller=seller_text)
    
    if buyer_accepts and seller_accepts:
        report_sale(
            prices=[buyer_bid[t], seller_bid[t]],
            elapsed_time=t+1,
            buyer_time_price=buyer_time_price,
            seller_time_price=seller_time_price,
            high=high,
            low=low,
            )
        return
    elif buyer_accepts:
        report_sale(
            prices=seller_bid[t],
            elapsed_time=t+1,
            buyer_time_price=buyer_time_price,
            seller_time_price=seller_time_price,
            high=high,
            low=low,
            )
        return
    elif seller_accepts:
        report_sale(
            prices=buyer_bid[t],
            elapsed_time=t+1,
            buyer_time_price=buyer_time_price,
            seller_time_price=seller_time_price,
            high=high,
            low=low,
            )
        return
    elif seller_rejects and buyer_rejects:
        report_no_deal(
            buyer_null_util,
            seller_null_util,
            elapsed_time=t+1,
            buyer_time_price=buyer_time_price,
            seller_time_price=seller_time_price,
            )
        return
    elif buyer_rejects: # one last chance for seller to accept
        if seller_acc_imm_util > seller_null_util:
            simulsay(buyer='...', seller='Okay, I accept your price!')
            report_sale(
                prices=buyer_bid[t],
                elapsed_time=t+1,
                buyer_time_price=buyer_time_price,
                seller_time_price=seller_time_price,
                high=high,
                low=low,
                )
            return
        else:
            report_no_deal(
                buyer_null_util, seller_null_util,
                elapsed_time=t+1,
                buyer_time_price=buyer_time_price,
                seller_time_price=seller_time_price,
                )
            return
    elif seller_rejects: # one last chance for buyer to accept
        if buyer_acc_imm_util > buyer_null_util:
            simulsay(seller='...', buyer='Okay, I accept your price!')
            report_sale(
                prices=buyer_bid[t],
                elapsed_time=t+1,
                buyer_time_price=buyer_time_price,
                seller_time_price=seller_time_price,
                high=high,
                low=low,
                )
            return
        else:
            report_no_deal(
                buyer_null_util,
                seller_null_util,
                elapsed_time=t+1,
                buyer_time_price=buyer_time_price,
                seller_time_price=seller_time_price,
                )
            return
    
    # nobody accepted or rejected, so haggle it out
    total_time = gap_between_opening_prices / (exp_buyer_time_price + exp_seller_time_price)
    convergent_price = exp_low + (buyer_time_price * total_time)
    narrate('skip ahead to t={} min'.format(total_time))
    report_sale(
            prices=convergent_price,
            elapsed_time=total_time,
            buyer_time_price=buyer_time_price,
            seller_time_price=seller_time_price,
            high=high,
            low=low,
            )
    return

In [8]:
buyer_box = widgets.VBox([widgets.Label('buyer:')] + list(buyer_sliders.values()))
seller_box = widgets.VBox([widgets.Label('seller:')] + list(seller_sliders.values()))
output_box = widgets.interactive_output(calculate, {k: v for k, v in dict(buyer_sliders, **seller_sliders).items()})

# return a widget with our controls and output
widgets.VBox([widgets.HBox([buyer_box, seller_box]), output_box])